In [1]:
import pandas as pd
import numpy as np

In [7]:
class GEN_data:
    def __init__(self, step_size=500, payoff_matrix = [[5,0],[10,3]], p_mutation = .02):
        #rules of Code: C = 0，D = 1
        self.step_size = step_size
        self.payoff_m = payoff_matrix
        self.p_mutation = p_mutation # Mutation: 0.02 probability random selection strategy, 0.98 probability update according to the established strategy mode
        pass

    def random_stratey(self):
        data = []
        for i in range(self.step_size):
            oppnent_stra = np.random.randint(2)
            focal_stra = np.random.randint(2)
            data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        return np.array(data)
    
    def ALLC_ALLD_stratey(self):
        data = []
        for i in range(self.step_size):
            oppnent_stra = 0 # oppnent ALLC
            focal_stra = 1 # focal ALLD
            if self.p_mutation <= round(0.1 * np.random.random(), 2):
                rand_oppnent_stra = np.random.randint(2); rand_focal_stra = np.random.randint(2)
                data.append([rand_oppnent_stra, rand_focal_stra, self.payoff_m[rand_oppnent_stra][rand_focal_stra], self.payoff_m[rand_focal_stra][rand_oppnent_stra]])
            else: data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        return np.array(data)
    
    def CtoALLD_DCAlt_stratey(self):
        data = []; oppnent_stra = 0; focal_stra = 1
        data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        for i in range(self.step_size - 1):
            oppnent_stra = 1 # oppnent C-to-ALLD
            if i % 2 == 0: focal_stra = 0 # focal DC_Alt
            else: focal_stra = 1
            if self.p_mutation <= round(0.1 * np.random.random(), 2):
                rand_oppnent_stra = np.random.randint(2); rand_focal_stra = np.random.randint(2)
                data.append([rand_oppnent_stra, rand_focal_stra, self.payoff_m[rand_oppnent_stra][rand_focal_stra], self.payoff_m[rand_focal_stra][rand_oppnent_stra]])
            else: data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        return np.array(data)
    
    def TFT_GRIM_stratey(self):
        # Round 1 both choose C
        data = []; oppnent_stra = 0; focal_stra = 0; flag = 0
        data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        # Round 2 random choose C or D
        oppnent_stra = np.random.randint(2); focal_stra = np.random.randint(2)
        data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        # After Round 3
        for i in range(self.step_size - 2):
            opp_temp = oppnent_stra # last Round oppnent
            oppnent_stra = focal_stra # oppnent TFT
            if opp_temp == 1: flag = 1
            if flag == 1: focal_stra = 1
            else: focal_stra = 0
            if self.p_mutation <= round(0.1 * np.random.random(), 2):
                rand_oppnent_stra = np.random.randint(2); rand_focal_stra = np.random.randint(2)
                data.append([rand_oppnent_stra, rand_focal_stra, self.payoff_m[rand_oppnent_stra][rand_focal_stra], self.payoff_m[rand_focal_stra][rand_oppnent_stra]])
            else: data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        return np.array(data)
    
    def PTFT_DTFT_stratey(self):
        # Round 1
        data = []; oppnent_stra = 1; focal_stra = np.random.randint(2)
        data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        # Round 2
        for i in range(self.step_size - 1):
            opp_temp = oppnent_stra
            oppnent_stra = focal_stra # oppnent D-TFT
            if focal_stra == opp_temp: focal_stra = 0
            else: focal_stra = 1 # focal PTFT
            if self.p_mutation <= round(0.1 * np.random.random(), 2):
                rand_oppnent_stra = np.random.randint(2); rand_focal_stra = np.random.randint(2)
                data.append([rand_oppnent_stra, rand_focal_stra, self.payoff_m[rand_oppnent_stra][rand_focal_stra], self.payoff_m[rand_focal_stra][rand_oppnent_stra]])
            else: data.append([oppnent_stra, focal_stra, self.payoff_m[oppnent_stra][focal_stra], self.payoff_m[focal_stra][oppnent_stra]])
        return np.array(data)

In [3]:
class LSTM:
    def __init__(self,ob_step,pre_step,size,lr):
        
        self.ob_step = ob_step
        self.pre_step = pre_step
        self.dim = size
        self.learn_rate = lr

    def d_tanh(self,x):
        return 1/np.cosh(x)

    def sigmod(self,x):
        return 1/(1+np.exp(-x))

    def d_sigmod(self,x):
        return self.sigmod(x) * (1-self.sigmod(x))

    def softmax(self,x):
        return np.exp(x)/np.exp(x).sum(axis=1).reshape(-1,1)

    # def d_softmax(self,x):
        # return self.sox * (1- x)

    def init_para(self):
        w_list = ['f_w','i_w','c_w','o_w']
        b_list = ['f_b','i_b','c_b','o_b']
        self.para={i:np.random.rand(sum(self.dim[:2]),self.dim[1])*2-1 for i in w_list}

        for i in b_list: 
            self.para[i] = np.random.rand(self.ob_step,self.dim[1])*2-1

        self.para['fnn_w1'] = np.random.rand(self.dim[1],2) *2-1
        self.para['fnn_b1'] = np.random.rand(self.ob_step,2) *2-1

        self.para['fnn_w2'] = np.random.rand(self.pre_step,self.ob_step) *2-1
        self.para['fnn_b2'] = np.random.rand(self.pre_step, 2) *2-1

        self.ht = np.random.rand(self.ob_step,self.dim[1])
        self.ct = np.random.rand(self.ob_step,self.dim[1])

        self.ht1 = np.random.rand(self.ob_step,self.dim[1])
        self.ct1 = np.random.rand(self.ob_step,self.dim[1])

    def predict(self,x,y):
        y = np.identity(2)[y]
        self.conc_x = np.concatenate([x,self.ht],axis=1)
    
        #forget gate
        
        self.F_in = np.dot(self.conc_x,self.para['f_w']) + self.para['f_b']
        self.F_out1 = self.sigmod(self.F_in)

        self.F_out2 = self.F_out1 * self.ct

        #input gate
        self.I_in = np.dot(self.conc_x,self.para['i_w'] ) + self.para['i_b'] 
        self.I_out1 = self.sigmod(self.I_in)
        
        self.C_in = np.dot(self.conc_x,self.para['c_w'] ) + self.para['c_b'] 
        self.C_out1 = np.tanh(self.C_in)

        self.I_out2 = self.I_out1 * self.C_out1
        
        #output gate
        self.O_in = np.dot(self.conc_x,self.para['o_w'] ) + self.para['o_b'] 
        self.O_out1 = self.sigmod(self.O_in)

        self.ct1 = self.F_out2 + self.I_out2
        self.ht1 = self.O_out1 * np.tanh(self.ct1)

        #FNN to predict y

        self.l1_in = np.dot(self.ht1, self.para['fnn_w1'] ) + self.para['fnn_b1'] 
        self.l1_out = np.tanh(self.l1_in)

        self.l2_in = np.dot(self.para['fnn_w2'],self.l1_out) + self.para['fnn_b2']

        # out prob of y, shape: [prob of C , prob of D ] * pre_step
        self.y_pred = self.softmax( self.l2_in - self.l2_in.max(axis=1).reshape(-1,1))

        self.loss = sum(-sum(y * np.log(self.y_pred)))

    def next_pred(self):
        self.ht = self.ht1
        self.ct = self.ct1

        pass
    def update_para(self,x,y):
        y = np.identity(2)[y]

        # FNN grade
        self.d_l_b2 = self.y_pred - y 
        self.d_l_w2 = np.dot(self.d_l_b2,self.l1_out.T)

        self.d_l_b1 = self.d_tanh(self.l1_in) * np.dot(self.para['fnn_w2'].T, self.d_l_b2)
        self.d_l_w1 = np.dot(self.ht1.T,self.d_l_b1)

        # LSTM grad
        self.d_f_ht1 = np.dot(self.d_l_b1,self.para['fnn_w1'].T)

        # forget gate 
        self.d_f_b = self.d_f_ht1 * self.O_out1 * self.d_tanh(self.ct1) * self.ct * self.d_sigmod(self.F_in)
        self.d_f_w = np.dot(self.conc_x.T,self.d_f_b)


        # input gate 
        temp_loss = self.d_f_ht1 * self.O_out1 * self.d_tanh(self.ct1)
        self.d_i_b = temp_loss  * self.C_out1 * self.d_sigmod(self.I_in)
        self.d_i_w =  np.dot(self.conc_x.T,self.d_i_b)

        self.d_c_b = temp_loss * self.I_out1 * self.d_tanh(self.C_in)
        self.d_c_w = np.dot(self.conc_x.T,self.d_c_b)

        # output gate
        self.d_o_b = self.d_f_ht1 * np.tanh(self.ct1) * self.d_sigmod(self.O_in) 
        self.d_o_w = np.dot(self.conc_x.T,self.d_o_b)

        w_list = ['f_w','i_w','c_w','o_w']
        b_list = ['f_b','i_b','c_b','o_b']
        dw_list = [self.d_f_w,self.d_i_w,self.d_c_w,self.d_o_w] 
        db_list = [self.d_f_b,self.d_i_b,self.d_c_b,self.d_o_b]
        for i,j in enumerate(w_list):
            self.para[j] += -self.learn_rate * dw_list[i]
            self.para[b_list[i]] += -self.learn_rate * db_list[i]
        self.para['fnn_w1'] += -self.learn_rate * self.d_l_w1
        self.para['fnn_b1'] += -self.learn_rate * self.d_l_b1

        self.para['fnn_w2'] += -self.learn_rate * self.d_l_w2
        self.para['fnn_b2'] += -self.learn_rate * self.d_l_b2



        pass

In [4]:
#简单测试网络LSTM反向传播

# [feature size ；output_gate]
# [4个特征[对手策略序列，自己策略序列，对方收益，己方收益]; LSTM神经元数目]
size = [4,70]

#观测步长
obverser_step = 10

#预测步长
predict_step = 3

x = np.random.rand(obverser_step,4)
y = list(np.random.randint(2,size=(predict_step)))
learn_rate = 0.01

test_case  = LSTM(obverser_step,predict_step,size,learn_rate)
test_case.init_para()

for i in range(20):
    test_case.predict(x,y)
    print('LOSS:%f' % test_case.loss)
    test_case.update_para(x,y)
    test_case.next_pred()
print(test_case.y_pred,y)


LOSS:3.436308
LOSS:3.228647
LOSS:2.389280
LOSS:1.494327
LOSS:1.169956
LOSS:1.231383
LOSS:2.096424
LOSS:3.289467
LOSS:4.095478
LOSS:3.528820
LOSS:3.104179
LOSS:2.416408
LOSS:1.773512
LOSS:1.071475
LOSS:0.655934
LOSS:0.591825
LOSS:0.558187
LOSS:0.566106
LOSS:1.015433
LOSS:1.001975
[[0.01137335 0.98862665]
 [0.79107102 0.20892898]
 [0.46946156 0.53053844]] [1, 0, 0]


In [5]:
# 使用随机策略生成的数据训练
get_data = GEN_data()
obverser_step = 10
predict_step = 5
size = [4,128]
laern_rate = 0.01

case = LSTM(obverser_step,predict_step,size,learn_rate)

case.init_para()
# training
for rod in range(300):
    data = get_data.random_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.update_para(temp_x,temp_y)
        case.next_pred()
        # print('LOSS:%f' % case.loss)


# testing
rights = 0
total_count = 0
for rod in range(10):
    data = get_data.random_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.next_pred()   
        rights += (temp_y == case.y_pred.argmax(axis=1)).sum()
        total_count += len(temp_y)


        # print('LOSS:%f' % case.loss)
print('预测精度：%f' % (rights/total_count))


<ipython-input-3-476a8d290a79>:10: RuntimeWarning: overflow encountered in cosh
  return 1/np.cosh(x)


预测精度：0.500828


In [6]:
# 使用TFT vs GRIM策略生成的数据训练
get_data = GEN_data()
obverser_step = 10
predict_step = 5
size = [4,128]
learn_rate = 0.01

case = LSTM(obverser_step,predict_step,size,learn_rate)

case.init_para()
# training
for rod in range(300):
    data = get_data.TFT_GRIM_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.update_para(temp_x,temp_y)
        case.next_pred()
        # print('LOSS:%f' % case.loss)


# testing
rights = 0
total_count = 0
for rod in range(10):
    data = get_data.TFT_GRIM_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.next_pred()   
        rights += (temp_y == case.y_pred.argmax(axis=1)).sum()
        total_count += len(temp_y)


        # print('LOSS:%f' % case.loss)
print('预测精度：%f' % (rights/total_count))


<ipython-input-3-476a8d290a79>:10: RuntimeWarning: overflow encountered in cosh
  return 1/np.cosh(x)


预测精度：0.577350


In [5]:
# 使用ALLC vs ALLD策略生成的数据训练
get_data = GEN_data()
obverser_step = 10
predict_step = 5
size = [4,128]
learn_rate = 0.01

case = LSTM(obverser_step,predict_step,size,learn_rate)

case.init_para()
# training
for rod in range(300):
    data = get_data.ALLC_ALLD_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.update_para(temp_x,temp_y)
        case.next_pred()
        # print('LOSS:%f' % case.loss)


# testing
rights = 0
total_count = 0
for rod in range(10):
    data = get_data.ALLC_ALLD_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.next_pred()   
        rights += (temp_y == case.y_pred.argmax(axis=1)).sum()
        total_count += len(temp_y)


        # print('LOSS:%f' % case.loss)
print('预测精度：%f' % (rights/total_count))

<ipython-input-3-476a8d290a79>:10: RuntimeWarning: overflow encountered in cosh
  return 1/np.cosh(x)


预测精度：0.578509


In [6]:
# 使用C-to-ALLD vs DC-Alt策略生成的数据训练
get_data = GEN_data()
obverser_step = 10
predict_step = 5
size = [4,128]
learn_rate = 0.01

case = LSTM(obverser_step,predict_step,size,learn_rate)

case.init_para()
# training
for rod in range(300):
    data = get_data.CtoALLD_DCAlt_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.update_para(temp_x,temp_y)
        case.next_pred()
        # print('LOSS:%f' % case.loss)


# testing
rights = 0
total_count = 0
for rod in range(10):
    data = get_data.CtoALLD_DCAlt_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.next_pred()   
        rights += (temp_y == case.y_pred.argmax(axis=1)).sum()
        total_count += len(temp_y)


        # print('LOSS:%f' % case.loss)
print('预测精度：%f' % (rights/total_count))

<ipython-input-3-476a8d290a79>:10: RuntimeWarning: overflow encountered in cosh
  return 1/np.cosh(x)


预测精度：0.575445


In [8]:
# 使用D-TFT vs PTFT生成的数据训练
get_data = GEN_data()
obverser_step = 10
predict_step = 5
size = [4,128]
laern_rate = 0.01

case = LSTM(obverser_step,predict_step,size,learn_rate)

case.init_para()
# training
for rod in range(300):
    data = get_data.PTFT_DTFT_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.update_para(temp_x,temp_y)
        case.next_pred()
        # print('LOSS:%f' % case.loss)


# testing
rights = 0
total_count = 0
for rod in range(10):
    data = get_data.PTFT_DTFT_stratey()
    for i in range(len(data[:,0]) - (obverser_step + predict_step+2) ):
        temp_x = data[i:i+obverser_step,:]
        temp_y = list(data[i+obverser_step:i+obverser_step+predict_step,0])

        case.predict(temp_x,temp_y)
        case.next_pred()   
        rights += (temp_y == case.y_pred.argmax(axis=1)).sum()
        total_count += len(temp_y)


        # print('LOSS:%f' % case.loss)
print('预测精度：%f' % (rights/total_count))

<ipython-input-3-476a8d290a79>:10: RuntimeWarning: overflow encountered in cosh
  return 1/np.cosh(x)


预测精度：0.517184
